In [40]:
import json
import os
import pandas as pd

In [41]:
# testing drugs dataset 

real_data = pd.read_csv("/Users/anshusingh/DPPCC/whitespace/benchmarking-synthetic-data-generators/drug_test.csv")
synthetic_data = pd.read_csv("/Users/anshusingh/DPPCC/whitespace/benchmarking-synthetic-data-generators/outputs/2023-11-18/synthcity/tabular/tvae_impute/drugs/drugs_imputed_data.csv")

In [42]:
from hyperimpute.plugins.imputers import Imputers

imputers = Imputers()

imputers.list()

['mean',
 'nop',
 'missforest',
 'hyperimpute',
 'sinkhorn',
 'most_frequent',
 'gain',
 'miwae',
 'sklearn_missforest',
 'mice',
 'sklearn_ice',
 'softimpute',
 'EM',
 'miracle',
 'median',
 'ice']

In [43]:
synthetic_data.head()

,Unnamed: 0,ID,Date,Date Type,Age,Sex,Race,Residence City,Residence County,Residence State,...,Morphine (Not Heroin),Hydromorphone,Xylazine,Other,Opiate NOS,Any Opioid,CardioCondition,RespiratoryCondition,ObesityCondition,DiabetesCondition
0,0,12-0187,07/17/2012,DATEOFDEATH,34,FEMALE,WHITE,MAHOPAC,PUTNAM,CT,...,Y,Y,Y,DUSTER,Y,Y,Y,Y,Y,Y
1,1,12-0258,10/01/2012,DATEOFDEATH,51,MALE,WHITE,PORTLAND,MIDDLESEX,CT,...,Y,Y,Y,KET,Y,Y,Y,Y,Y,Y
2,2,13-0146,04/28/2013,DATEOFDEATH,28,MALE,WHITE,MASTIC BEACH,HILLSBOROUGH,CT,...,Y,Y,Y,MORPHINE,Y,Y,Y,Y,Y,Y
3,3,14-0150,04/06/2014,DATEOFDEATH,46,MALE,WHITE,WATERBURY,LITCHFIELD,CT,...,Y,Y,Y,METHOXYPCP,Y,Y,Y,Y,Y,Y
4,4,14-0183,04/27/2014,DATEOFDEATH,52,MALE,WHITE,NEW LONDON,ONSLOW,CA,...,Y,Y,Y,HYROMORPH,Y,Y,Y,Y,Y,Y


In [44]:
real_data.head()

,ID,Date,Date Type,Age,Sex,Race,Residence City,Residence County,Residence State,Death City,...,Morphine (Not Heroin),Hydromorphone,Xylazine,Other,Opiate NOS,Any Opioid,CardioCondition,RespiratoryCondition,ObesityCondition,DiabetesCondition
0,12-0187,07/17/2012,DATEOFDEATH,34.0,FEMALE,WHITE,MAHOPAC,PUTNAM,NaN,DANBURY,...,NaN,NaN,NaN,DUSTER,NaN,NaN,NaN,NaN,NaN,NaN
1,12-0258,10/01/2012,DATEOFDEATH,51.0,MALE,WHITE,PORTLAND,MIDDLESEX,NaN,PORTLAND,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,13-0146,04/28/2013,DATEOFDEATH,NaN,MALE,WHITE,NaN,NaN,NaN,HARTFORD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,14-0150,04/06/2014,DATEOFDEATH,46.0,MALE,WHITE,WATERBURY,NaN,NaN,TORRINGTON,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,14-0183,04/27/2014,DATEOFDEATH,52.0,MALE,WHITE,NEW LONDON,NaN,NaN,NEW LONDON,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
synthetic_data.drop(columns=['Unnamed: 0'], inplace=True)

In [35]:
synthetic_data.columns

Index(['ID', 'Date', 'Date Type', 'Age', 'Sex', 'Race', 'Residence City',
       'Residence County', 'Residence State', 'Death City', 'Death County',
       'Location', 'Location if Other', 'Injury Place', 'Injury City',
       'Injury County', 'Injury State', 'Other Significant Conditions ',
       'Heroin', 'Cocaine', 'Fentanyl', 'Fentanyl Analogue', 'Oxycodone',
       'Oxymorphone', 'Ethanol', 'Hydrocodone', 'Benzodiazepine', 'Methadone',
       'Amphet', 'Tramad', 'Morphine (Not Heroin)', 'Hydromorphone',
       'Xylazine', 'Other', 'Opiate NOS', 'Any Opioid', 'CardioCondition',
       'RespiratoryCondition', 'ObesityCondition', 'DiabetesCondition'],
      dtype='object')

In [36]:
# Function to compare datasets
def compare_datasets(real_df, synthetic_df):
    if real_df.columns.tolist() != synthetic_df.columns.tolist():
        return "The datasets do not have the same columns"

    for column in real_df.columns:
        # Filter out null values
        real_unique = set(real_df[column].dropna().unique())
        synthetic_unique = set(synthetic_df[column].dropna().unique())

        if len(real_unique) != len(synthetic_unique):
            return f"Column '{column}' does not have the same number of unique values"

        if real_unique != synthetic_unique:
            return f"Column '{column}' does not have the same domain of unique values"

    return "Both datasets have the same number and domain of unique values for each column"

# Compare the datasets
comparison_result = compare_datasets(real_data, synthetic_data)
print(comparison_result)


Both datasets have the same number and domain of unique values for each column


In [26]:
# Function to find columns with missing values
def find_missing_values_columns(dataframe):
    num_cols_with_missing = []
    non_num_cols_with_missing = []

    # Iterate through each column
    for col in dataframe.columns:
        # Check if the column has missing values
        if dataframe[col].isna().any():
            # Check if the column is numeric
            if pd.api.types.is_numeric_dtype(dataframe[col]):
                num_cols_with_missing.append(col)
            else:
                non_num_cols_with_missing.append(col)
    
    return num_cols_with_missing, non_num_cols_with_missing

# Get the columns with missing values
numeric_columns, non_numeric_columns = find_missing_values_columns(real_data)

print("Numeric columns with missing values:", numeric_columns)
print("Number of numeric columns with missing values:", len(numeric_columns))
print("Non-numeric columns with missing values:", non_numeric_columns)
print("Number of non-numeric columns with missing values:", len(non_numeric_columns))


Numeric columns with missing values: ['Age']
Number of numeric columns with missing values: 1
Non-numeric columns with missing values: ['Sex', 'Race', 'Residence City', 'Residence County', 'Residence State', 'Death City', 'Death County', 'Location', 'Location if Other', 'Injury Place', 'Injury City', 'Injury County', 'Injury State', 'Other Significant Conditions ', 'Heroin', 'Cocaine', 'Fentanyl', 'Fentanyl Analogue', 'Oxycodone', 'Oxymorphone', 'Ethanol', 'Hydrocodone', 'Benzodiazepine', 'Methadone', 'Amphet', 'Tramad', 'Morphine (Not Heroin)', 'Hydromorphone', 'Xylazine', 'Other', 'Opiate NOS', 'Any Opioid', 'CardioCondition', 'RespiratoryCondition', 'ObesityCondition', 'DiabetesCondition']
Number of non-numeric columns with missing values: 36


In [12]:
# Compare the datasets
comparison_result = compare_datasets(real_data, synthetic_data)
print(comparison_result)

Column 'Sex' does not have the same number of unique values


In [3]:
real.columns

Index(['ID', 'Date', 'Date Type', 'Age', 'Sex', 'Race', 'Residence City',
       'Residence County', 'Residence State', 'Death City', 'Death County',
       'Location', 'Location if Other', 'Injury Place', 'Injury City',
       'Injury County', 'Injury State', 'Other Significant Conditions ',
       'Heroin', 'Cocaine', 'Fentanyl', 'Fentanyl Analogue', 'Oxycodone',
       'Oxymorphone', 'Ethanol', 'Hydrocodone', 'Benzodiazepine', 'Methadone',
       'Amphet', 'Tramad', 'Morphine (Not Heroin)', 'Hydromorphone',
       'Xylazine', 'Other', 'Opiate NOS', 'Any Opioid', 'CardioCondition',
       'RespiratoryCondition', 'ObesityCondition', 'DiabetesCondition'],
      dtype='object')

In [5]:
imputed.columns

Index(['Unnamed: 0', 'ID', 'Date', 'Date Type', 'Age', 'Sex', 'Race',
       'Residence City', 'Residence County', 'Residence State', 'Death City',
       'Death County', 'Location', 'Location if Other', 'Injury Place',
       'Injury City', 'Injury County', 'Injury State',
       'Other Significant Conditions ', 'Heroin', 'Cocaine', 'Fentanyl',
       'Fentanyl Analogue', 'Oxycodone', 'Oxymorphone', 'Ethanol',
       'Hydrocodone', 'Benzodiazepine', 'Methadone', 'Amphet', 'Tramad',
       'Morphine (Not Heroin)', 'Hydromorphone', 'Xylazine', 'Other',
       'Opiate NOS', 'Any Opioid', 'CardioCondition', 'RespiratoryCondition',
       'ObesityCondition', 'DiabetesCondition'],
      dtype='object')

In [11]:
from sdv.datasets.demo import get_available_demos, download_demo

In [5]:
from ydata_profiling import ProfileReport

/Users/anshusingh/.pyenv/versions/3.8.11/lib/python3.8/site-packages/numba/core/decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)
/Users/anshusingh/.pyenv/versions/3.8.11/lib/python3.8/site-packages/visions/backends/shared/nan_handling.py:51: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def hasna(x: np.ndarray) -> bool:


In [5]:
datasets_df = get_available_demos(modality='single_table')
datasets_df

,dataset_name,size_MB,num_tables
0,KRK_v1,0.07,1
1,adult,3.91,1
2,alarm,4.52,1
3,asia,1.28,1
4,census,98.17,1
5,census_extended,4.95,1
6,child,3.20,1
7,covtype,255.65,1
8,credit,68.35,1
9,expedia_hotel_logs,0.20,1


In [6]:
single_table_dataset_names = list(datasets_df["dataset_name"])

In [9]:
def get_dataset_with_sdv(modality, dataset_name):
    real_data, metadata = download_demo(
        modality=modality,
        dataset_name=dataset_name,
)

In [6]:
def get_minimal_profile(variables_dict):
    profiles = {}
    profiles["table"] = variables_dict["table"]
    profiles["alerts"] = variables_dict["alerts"]
    profiles["variables"] = {}
    for k, v in variables_dict["variables"].items():
        dtype = v["type"]
        var_profile = {}
        
        # Supported Ydata types
        # - Boolean
        # - Numerical
        # - Date (and Datetime)
        # - Categorical
        # - Time-series
        # - URL
        # - Path
        # - File
        # - Image
        if dtype == "Categorical":
            var_profile = {
                "type": dtype,
                "n_distinct": v["n_distinct"],
                "n_missing": v["n_missing"],
                "freq_counts": v["word_counts"],
                "imbalance": v["imbalance"]
            }
        elif dtype == "Numeric":
            var_profile = {
                "type": dtype,
                "n_distinct": v["n_distinct"],
                "n_missing": v["n_missing"],             
                "kurtosis": v["kurtosis"],
                "skewness": v["skewness"],
                "monotonic": v["monotonic"]
            }
        elif dtype == "Boolean":
            var_profile = {
                "type": dtype,
                "n_distinct": v["n_distinct"],
                "n_missing": v["n_missing"],
                "imbalance": v["imbalance"]
            }
        else:    
            var_profile = v
            print("None a valid type : ", dtype)
        profiles["variables"][k] = var_profile
    return profiles

In [ ]:
for dataset_name in single_table_dataset_names:
    print("-*-"*10)
    print(" "*4, dataset_name)
    print("-*-"*10)
    
    output_path = f"dataset_profiles/{dataset_name}"
    if not os.path.exists(output_path):
        os.makedirs(output_path)
        
    df = pd.read_csv(f"data/tabular/{dataset_name}.csv")
    profile = ProfileReport(df, minimal=False)
    
    # As a JSON string
    json_data = profile.to_json()
        
    json_dict = json.loads(json_data)
    var_profiles = get_minimal_profile(json_dict)
        
    # Save the dictionary as a JSON file
    with open(f"{output_path}/{dataset_name}.json", "w") as json_file:
        json.dump(var_profiles, json_file)

    # As a file
    profile.to_file(f"{output_path}/{dataset_name}_detailed.json")
    profile.to_file(f"{output_path}/{dataset_name}.html")

In [3]:
seq_datasets_df = get_available_demos(modality='sequential')
seq_table_dataset_names = seq_datasets_df["dataset_name"]

In [4]:
seq_datasets_df

,dataset_name,size_MB,num_tables
0,ArticularyWordRecognition,8.61,1
1,AtrialFibrillation,0.92,1
2,BasicMotions,0.64,1
3,CharacterTrajectories,19.19,1
4,Cricket,17.24,1
5,DuckDuckGeese,291.38,1
6,ERing,1.25,1
7,EchoNASDAQ,4.16,1
8,EigenWorms,372.63,1
9,Epilepsy,3.17,1


In [ ]:
for dataset_name in seq_table_dataset_names:
    print("-*-"*10)
    print(" "*4, dataset_name)
    print("-*-"*10)
    
    try:
        output_path = f"dataset_profiles/seq/{dataset_name}"
        if not os.path.exists(output_path):
            os.makedirs(output_path)
        else:
            continue

        df = pd.read_csv(f"data/sequential/{dataset_name}.csv")
        profile = ProfileReport(df, minimal=True)

        # As a JSON string
        json_data = profile.to_json()

        json_dict = json.loads(json_data)
        var_profiles = get_minimal_profile(json_dict)

        # Save the dictionary as a JSON file
        with open(f"{output_path}/{dataset_name}.json", "w") as json_file:
            json.dump(var_profiles, json_file)

        # As a file
        profile.to_file(f"{output_path}/{dataset_name}_detailed.json")
        profile.to_file(f"{output_path}/{dataset_name}.html")
    except Exception as e:
        print(e)